In [ ]:
%%writefile mixed_gausian_mcmc.py

# ライブラリ読み込み
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
import scipy.special
import os
import pandas as pd

# 関数の定義
def gaussian(x, mu, sigma):
    """1次元ガウス分布の確率密度関数"""
    return 1 / (sigma * np.sqrt(2 * np.pi)) * np.exp(-(x - mu)**2 / (2 * sigma**2))

def log_gaussian(x, mu, sigma):
    """1次元ガウス分布の正規確率密度関数の対数"""
    return -1/2 * np.log(2*np.pi) -np.log(sigma) -((x - mu)**2 / (2 * sigma**2))
 
def invgamma(x, alpha, beta):
    """逆ガンマ分布の確率密度関数"""
    return beta**alpha / scipy.special.gamma(alpha) * x**(-alpha - 1) * np.exp(-beta / x)

def gibbs_sampling(X, K=2, iterations=1000, mu0=0, nu0=0.01, alpha0=2, beta0=1):
    """ギブスサンプリングの実装"""
    N = len(X)

    # 事前分布のパラメータ
    gamma = np.ones(K)  # 混合比率の事前分布の濃度パラメータ

    # パラメータの初期化
    mu = np.random.normal(mu0, 1 / np.sqrt(nu0), K)
    sigma2 = 1 / np.random.gamma(alpha0, 1 / beta0, K)
    pi = np.random.dirichlet(gamma)
    z = np.random.choice(K, N)

    # パラメータの履歴を保存するためのリスト
    mu_history = []
    sigma2_history = []
    pi_history = []
   
    # グラフ表示するためのパラメータ
    # 混合正規分布を計算するためのxの範囲
    x = np.linspace(-50, 50, 1000)
    current_directory = os.getcwd()
    output_folder = os.path.join(current_directory, "images")

    # 画像を置換する処理のためのおまじない
    image_output = st.empty()
   
    # ギブスサンプリングの繰り返し
    for iteration in range(iterations):
        # zの更新
        for n in range(N):
            log_p = np.log(pi) + log_gaussian(X[n], mu, np.sqrt(sigma2))
            log_p_shifted = log_p - scipy.special.logsumexp(log_p)
            p = np.exp(log_p_shifted)
            z[n] = np.random.choice(K, p=p)

        # muの更新
        for k in range(K):
            Nk = np.sum(z == k)
            xbar = np.sum(X[z == k]) / Nk if Nk > 0 else 0
            mun = (nu0 * mu0 + Nk * xbar) / (nu0 + Nk)
            nun = nu0 + Nk
            mu[k] = np.random.normal(mun, 1 / np.sqrt(nun))

        # sigma2の更新
        for k in range(K):
            Nk = np.sum(z == k)
            ss = np.sum((X[z == k] - mu[k])**2) if Nk > 0 else 0
            alphan = alpha0 + Nk / 2
            betan = beta0 + ss / 2
            sigma2[k] = 1 / np.random.gamma(alphan, 1 / betan)

        # piの更新
        pi = np.random.dirichlet(gamma + np.bincount(z, minlength=K))

        # パラメータの履歴を保存
        mu_history.append(mu.copy())
        sigma2_history.append(sigma2.copy())
        pi_history.append(pi.copy())

        # 混合正規分布の確率密度関数
        pdf = 0
        for k in range(K):
            pdf += pi[k] * gaussian(x, mu[k], np.sqrt(sigma2[k]))

        plt.plot(x, pdf, label=f'1次元混合正規分布{iteration}番目')

        # ヒストグラムを経験分布としてプロット
        plt.hist(X, bins=100, density=True, alpha=0.5, label='データのヒストグラム')

        plt.xlabel('x')
        plt.ylabel('確率密度')
        plt.title('1次元混合正規分布とデータのヒストグラム')
        plt.legend()
        plt.grid(alpha=0.4)
        # 画像を保存
        image_file = f"iteration_{iteration}.png"
        filename = os.path.join(output_folder, image_file)
        plt.savefig(filename)
        image_path = os.path.join(output_folder, image_file)
        image_output.image(image_path, caption=f"Iteration:{iteration}", use_column_width=True)
        # st.image(image_path, caption=f"Iteration:{iteration}", use_column_width=True)
        plt.close()

        # 学習停止ボタンの処理
        if st.session_state.stop_learning:
            break  # ループを中断

    # リストをNumPy配列に変換
    mu_history = np.array(mu_history)
    sigma2_history = np.array(sigma2_history)
    pi_history = np.array(pi_history)

    return mu, np.sqrt(sigma2), pi, z, mu_history, sigma2_history, pi_history

# StreamLit
st.title("混合正規分布モデルをMCMCで実装してみた")

st.subheader("自分のデータを使う？　サンプルデータ作成する？")
selected_item = st.radio('どっちがいい?',['自分のデータ', 'サンプルデータ作成'])

# 学習停止ボタンの初期化
stop_button_placeholder = st.empty()
if 'stop_learning' not in st.session_state:
    st.session_state.stop_learning = False

if selected_item == "自分のデータ":
    # CSVファイルのアップロード
    uploaded_file = st.file_uploader("CSVファイルを以下の書式に合わせてアップロードしてください", type=["csv"])
    current_directory = os.getcwd()
    output_folder = os.path.join(current_directory, "images")
    image_path = os.path.join(output_folder, "csv_sample.png")
    st.image(image_path, caption = "csvファイル例")
   
    # アップロードされたファイルをデータフレームに読み込む
    if uploaded_file is not None:
        X = np.loadtxt(uploaded_file, delimiter=",")

        st.write("アップロードされたデータ", X)

        st.subheader("混合数を決めるよ")
        K = st.number_input("混合数", value=1)
       
        st.subheader("以下でハイパーパラメータを調整")
        mu0 = st.number_input("平均の事前分布の平均", value=0.0)  # 平均の事前分布の平均
        nu0 = st.number_input("平均の事前分布の精度", value=0.01)  # 平均の事前分布の精度
        alpha0 = st.number_input("分散の事前分布の形状パラメータ", value=2)  # 分散の事前分布の形状パラメータ
        beta0 = st.number_input("分散の事前分布の尺度パラメータ", value=1)  # 分散の事前分布の尺度パラメータ
       
        st.subheader("学習の反復回数を決めるよ")
        iterations = st.number_input("iterations", value=1000)

        if st.button('ギブスサンプリングを実行'):
            # セッション状態の初期化
            if 'stop_learning' not in st.session_state:
                st.session_state.stop_learning = False
            if 'results_df' not in st.session_state:
                st.session_state.results_df = None
            if 'fig' not in st.session_state:
                st.session_state.fig = None

                        
            # 学習停止ボタンの描画
            if not st.session_state.stop_learning:
                if stop_button_placeholder.button('学習停止'):
                    st.session_state.stop_learning = True
                    st.experimental_rerun()  # ボタンの状態が変わったので再実行
            
            mu, sig, pi, z, mu_history, sigma2_history, pi_history = gibbs_sampling(X=X, K=K, iterations=iterations, mu0=mu0, nu0=nu0, alpha0=alpha0, beta0=beta0)
            
            # 結果出力部分
            st.subheader("結果発表🥳")
            if st.session_state.stop_learning:
                st.write("学習が途中で停止されました。現時点までの結果:")
                st.session_state.stop_learning = False  # リセット
            else:
                st.write("学習完了！最終的な結果:")
            results_df = pd.DataFrame({
                "推定された値": [mu, sig, pi]
            }, index=["平均", "標準偏差", "混合比率"])
   
            st.write(results_df)

            # パラメータの時系列プロット
            st.subheader("パラメータの時系列プロット")
            fig, axes = plt.subplots(3, 1, figsize=(10, 12))

            for k in range(K):
                axes[0].plot(mu_history[:, k], label=f"mu{k+1}")
                axes[1].plot(np.sqrt(sigma2_history[:, k]), label=f"sigma{k+1}")
                axes[2].plot(pi_history[:, k], label=f"pi{k+1}")
        
            axes[0].set_ylabel("mu")
            axes[1].set_ylabel("sigma")
            axes[2].set_ylabel("pi")
            axes[0].legend()
            axes[1].legend()
            axes[2].legend()
        
            plt.xlabel("iteration")
            st.pyplot(fig)                

elif selected_item == "サンプルデータ作成":
    st.write("どういうデータを作る？")
    K = st.number_input("混合数", value=1)
    mean = []
    sigma = []
    ratios = []
    for k in range(K):
        u = st.number_input(f"{k+1}番目の平均パラメータ", value=5.0)
        s = st.number_input(f"{k+1}番目の標準偏差パラメータ", value=1.0)
        ratio = st.number_input(f"混合割合 {k+1} を入力してください (0~1)", min_value=0.0, max_value=1.0, step=0.01)
        mean.append(u)
        sigma.append(s)
        ratios.append(ratio)
   
    if sum(ratios) != 1.0:
        st.error("混合割合の合計が1ではありません。")
    else:
        st.write(f"混合数:{K}")
        st.write(f"混合割合:{ratios}")
        st.write(f"平均パラメータ:{mean}")
        st.write(f"標準偏差パラメータ:{sigma}")
   
        # データ数入力
        N = st.number_input("データ数", value=1000)
   
        # データ生成
        z = np.random.choice(K, size=N, p=ratios)  # 各データがどの正規分布に属するかを決定
        X = np.zeros(N)
        for i in range(N):
            X[i] = np.random.normal(mean[z[i]], sigma[z[i]])
   
        st.write("生成されたデータ", X)

        st.write("以下でハイパーパラメータを調整")
        mu0 = st.number_input("平均の事前分布の平均", value=0.0)  # 平均の事前分布の平均
        nu0 = st.number_input("平均の事前分布の精度", value=0.01)  # 平均の事前分布の精度
        alpha0 = st.number_input("分散の事前分布の形状パラメータ", value=2)  # 分散の事前分布の形状パラメータ
        beta0 = st.number_input("分散の事前分布の尺度パラメータ", value=1)  # 分散の事前分布の尺度パラメータ
        st.write("学習の反復回数を決めるよ")
        iterations = st.number_input("iterations", value=1000)

        if st.button('ギブスサンプリングを実行'):
            # セッション状態の初期化
            if 'stop_learning' not in st.session_state:
                st.session_state.stop_learning = False
            if 'results_df' not in st.session_state:
                st.session_state.results_df = None
            if 'fig' not in st.session_state:
                st.session_state.fig = None
            mu, sig, pi, z, mu_history, sigma2_history, pi_history = gibbs_sampling(X=X, K=K, iterations=iterations, mu0=mu0, nu0=nu0, alpha0=alpha0, beta0=beta0)
            # 結果出力部分
            st.subheader("結果発表🥳")
            if st.session_state.stop_learning:
                st.write("学習が途中で停止されました。現時点までの結果:")
                st.session_state.stop_learning = False  # リセット
            else:
                st.write("学習完了！最終的な結果:")
            results_df = pd.DataFrame({
                "真の値": [mean, sigma, ratios],
                "推定された値": [mu, sig, pi]
            }, index=["平均", "標準偏差", "混合比率"])
   
            st.write(results_df)

            # パラメータの時系列プロット
            st.subheader("パラメータの時系列プロット")
            fig, axes = plt.subplots(3, 1, figsize=(10, 12))

            for k in range(K):
                axes[0].plot(mu_history[:, k], label=f"mu{k+1}")
                axes[1].plot(np.sqrt(sigma2_history[:, k]), label=f"sigma{k+1}")
                axes[2].plot(pi_history[:, k], label=f"pi{k+1}")

            axes[0].set_ylabel("mu")
            axes[1].set_ylabel("sigma")
            axes[2].set_ylabel("pi")
            axes[0].legend()
            axes[1].legend()
            axes[2].legend()

            plt.xlabel("iteration")
            st.pyplot(fig)

In [ ]:
# アプリの起動
!echo | streamlit run mixed_gausian_mcmc.py